In [1]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF(r'F:\reviews_Apps_for_Android_5.json.gz')

In [2]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1N4O8VOJZTDVB,B004A9SDD8,Annette Yancey,"[1, 1]","Loves the song, so he really couldn't wait to ...",3.0,Really cute,1383350400,"11 2, 2013"
1,A2HQWU6HUKIEC7,B004A9SDD8,"Audiobook lover ""Kathy""","[0, 0]","Oh, how my little grandson loves this app. He'...",5.0,2-year-old loves it,1323043200,"12 5, 2011"
2,A1SXASF6GYG96I,B004A9SDD8,Barbara Gibbs,"[0, 0]",I found this at a perfect time since my daught...,5.0,Fun game,1337558400,"05 21, 2012"
3,A2B54P9ZDYH167,B004A9SDD8,"Brooke Greenstreet ""Babylove""","[3, 4]",My 1 year old goes back to this game over and ...,5.0,We love our Monkeys!,1354752000,"12 6, 2012"
4,AFOFZDTX5UC6D,B004A9SDD8,C. Galindo,"[1, 1]",There are three different versions of the song...,5.0,This is my granddaughters favorite app on my K...,1391212800,"02 1, 2014"


In [3]:
df = df.drop(['reviewerID','asin','reviewerName','helpful','summary','unixReviewTime','reviewTime'], axis=1)

In [4]:
df['label'] = 2

In [5]:
df.head()

,reviewText,overall,label
0,"Loves the song, so he really couldn't wait to ...",3.0,2
1,"Oh, how my little grandson loves this app. He'...",5.0,2
2,I found this at a perfect time since my daught...,5.0,2
3,My 1 year old goes back to this game over and ...,5.0,2
4,There are three different versions of the song...,5.0,2


In [6]:
df.loc[df['overall'] > 3, 'label'] = 1
df.loc[df['overall'] < 3, 'label'] = -1
df.loc[df['overall'] == 3, 'label'] = 0

In [7]:
df.head()

,reviewText,overall,label
0,"Loves the song, so he really couldn't wait to ...",3.0,0
1,"Oh, how my little grandson loves this app. He'...",5.0,1
2,I found this at a perfect time since my daught...,5.0,1
3,My 1 year old goes back to this game over and ...,5.0,1
4,There are three different versions of the song...,5.0,1


In [8]:
df.describe()

,overall,label
count,752937.000000,752937.000000
mean,3.968931,0.559967
std,1.342484,0.757222
min,1.000000,-1.000000
25%,3.000000,0.000000
50%,5.000000,1.000000
75%,5.000000,1.000000
max,5.000000,1.000000


In [9]:
train=df.sample(frac=0.8,random_state=200)
test=df.drop(train.index)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.reviewText)
X_train_counts.shape

(602350, 142967)

In [11]:
print(X_train_counts[0])

  (0, 90107)	1
  (0, 129752)	1
  (0, 42667)	1
  (0, 69607)	1
  (0, 50092)	1
  (0, 78294)	1
  (0, 82550)	1
  (0, 90978)	1
  (0, 86106)	1
  (0, 97200)	1
  (0, 88684)	1
  (0, 60324)	1
  (0, 35816)	1
  (0, 85225)	1
  (0, 127143)	1
  (0, 53080)	2
  (0, 52061)	1
  (0, 69233)	1
  (0, 125135)	2


In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(602350, 142967)

In [13]:
print(X_train_tfidf[0])

  (0, 125135)	0.152278469338
  (0, 69233)	0.0864892881435
  (0, 52061)	0.124414831908
  (0, 53080)	0.178471293758
  (0, 127143)	0.189817453598
  (0, 85225)	0.179751608206
  (0, 35816)	0.420917101451
  (0, 60324)	0.120157032765
  (0, 88684)	0.128799649057
  (0, 97200)	0.209706466011
  (0, 86106)	0.549578000815
  (0, 90978)	0.152252759426
  (0, 82550)	0.270508149967
  (0, 78294)	0.247142812925
  (0, 50092)	0.105990903679
  (0, 69607)	0.0710632435922
  (0, 42667)	0.204167466246
  (0, 129752)	0.270389257188
  (0, 90107)	0.104563708971


In [14]:
X_test_counts = count_vect.transform(test.reviewText)
X_test_counts.shape

(150587, 142967)

In [15]:
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train.label)

In [17]:
predicted = clf.predict(X_test_tfidf)

In [21]:
from sklearn import metrics

In [26]:
print(metrics.classification_report(test.label, predicted))

             precision    recall  f1-score   support

         -1       0.88      0.17      0.29     24695
          0       0.26      0.00      0.00     16956
          1       0.75      1.00      0.85    108936

avg / total       0.71      0.75      0.66    150587

